In [1]:
import pandas as pd
import numpy as np


df = pd.read_csv('input.csv')


df.columns = [col.strip() for col in df.columns]

date_columns = ['Date of Joining', 'Date of Exit', 'Compensation 1 date', 
                'Compensation 2 date', 'Review 1 date', 'Review 2 date', 
                'Engagement 1 date', 'Engagement 2 date']
for col in date_columns:
    df[col] = pd.to_datetime(df[col])

df.fillna(method='ffill', inplace=True)

def derive_dates(row):
    if pd.isnull(row['Date of Exit']):
        end_date = pd.Timestamp('2100-01-01')
    else:
        end_date = row['Date of Exit'] - pd.Timedelta(days=1)
    return pd.Series({'Effective Date': row['Date of Joining'], 'End Date': end_date})


date_df = df.apply(derive_dates, axis=1)


df = pd.concat([df, date_df], axis=1)


def transform_data(row):
    records = []
    for i in range(1, 3):
        record = {
            'Employee Code': row['Employee Code'],
            'Effective Date': row['Effective Date'],
            'End Date': row['End Date'],
            'Compensation': row['Compensation'],
            'Review': row[f'Review {i}'],
            'Engagement': row[f'Engagement {i}']
        }
        records.append(record)
    return pd.DataFrame(records)


transformed_df = pd.concat(df.apply(transform_data, axis=1).tolist(), ignore_index=True)

transformed_df.sort_values(by=['Employee Code', 'Effective Date'], inplace=True)


transformed_df.to_csv('historical_employee_data.csv', index=False)


print("Data transformation completed. Output saved to 'historical_employee_data.csv'.")


Data transformation completed. Output saved to 'historical_employee_data.csv'.
